In [45]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.getOrCreate()

In [44]:
df = spark.createDataFrame([(1, [20.0, 4.0, 2.0, 6.0, 10.0])], ("id", "values"))
df.select(f.aggregate("values", lit(0.0), lambda acc, x: acc + x).alias("sum")).show()

In [40]:
df = spark.createDataFrame([(1, [1, 2, 3, 4])], ("key", "values"))
df.select(transform("values", lambda x: x * 2).alias("doubled")).show()

In [46]:
def merge(acc, x):
     count = acc.count + 1
     sum = acc.sum + x
     return struct(count.alias("count"), sum.alias("sum"))
df.select(
    f.aggregate(
        "values",
         struct(lit(0).alias("count"), lit(0.0).alias("sum")),
         merge,
         lambda acc: acc.sum / acc.count,
     ).alias("mean")
 ).show()

Filter

In [47]:
df = spark.createDataFrame(
     [(1, ["2018-09-20",  "2019-02-03", "2019-07-01", "2020-06-01"])],
     ("key", "values")
)
def after_second_quarter(x):
     return month(to_date(x)) > 6
df.select(
     filter("values", after_second_quarter).alias("after_second_quarter")
).show(truncate=False)

In [33]:
df = spark.createDataFrame(
     [(1, ["bar"]), (2, ["foo", "bar"]), (3, ["foobar", "foo"])],
     ("key", "values")
 )
df.select(f.forall("values", lambda x: x.rlike("foo")).alias("all_foo")).show()

In [41]:
df = spark.createDataFrame([(1, [1, 2, 3, 4]), (2, [3, -1, 0])],("key", "values"))
df.select(exists("values", lambda x: x < 0).alias("any_negative")).show()

In [42]:
df = spark.createDataFrame([(1, [1, 2, 3, 4])], ("key", "values"))
df.select(transform("values", lambda x: x * 2).alias("doubled")).show()